###  Training


In [1]:
import quandl
q_api_key = "bB4wp5--7XrkpGZ7-gxJ"
quandl.ApiConfig.api_key = q_api_key
#Stocks=['GE', 'AMD', 'F', 'AAPL', 'TWTR', 'CHK', 'MU', 'MSFT', 'CSCO', 'T', 'SNAP', 'INTC', 'WFC', 'VALE', 'PFE', 'SWN', 'NVDA', 'WFT', 'CMCSA', 'FCX', 'SIRI', 'KMI', 'XOM', 'PBR', 'RAD', 'JPM', 'VZ', 'NOK', 'C', 'ABEV', 'RIG', 'NWL', 'ORCL', 'QCOM', 'VIPS', 'KO', 'AMAT', 'TEVA', 'AKS', 'ESV', 'FEYE', 'ABX', 'SLB', 'GM', 'CTL', 'SBUX', 'GRPN', 'CX', 'DAL', 'CBL', 'PG', 'RF', 'S', 'ATVI', 'MRK', 'JD', 'MGM', 'HAL', 'MRO', 'V', 'EXPE', 'HBI', 'FOXA', 'CVS', 'HPE', 'KEY', 'NBR', 'ECA', 'EBAY', 'FDC', 'MS', 'GG', 'AIG', 'JNJ', 'CZR', 'AUY', 'DDR', 'SAN', 'PYPL', 'CLF', 'WMT', 'ITUB', 'AMZN', 'MDLZ', 'GILD', 'NKE', 'BRX', 'PBR', 'A', 'KGC', 'HPQ', 'X', 'DWDP', 'ON', 'VER', 'RRC', 'CY', 'TSLA', 'SCHW', 'PTEN']
#Stocks=['GE', 'AMD', 'F','AAPL', 'TWTR', 'CHK']
#Stocks=['GE', 'AMD', 'F', 'AAPL', 'CHK', 'MU', 'AIG','MSFT', 'CSCO', 'T']# 'SNAP', 'INTC', 'WFC', 'VALE', 'PFE', 'SWN', 'NVDA', 'WFT', 'CMCSA', 'FCX', 'SIRI', 'KMI', 'XOM', 'PBR', 'RAD', 'JPM', 'VZ', 'NOK', 'C', 'ABEV', 'RIG', 'NWL', 'ORCL', 'QCOM', 'VIPS', 'KO', 'AMAT', 'TEVA', 'AKS', 'ESV', 'FEYE', 'ABX', 'SLB', 'GM', 'CTL', 'SBUX', 'GRPN', 'CX', 'DAL']

#df = quandl.get_table('WIKI/PRICES', ticker=Stocks, qopts = { 'columns': ['ticker', 'volume','adj_close'] }, date = { 'gte': '2011-12-31', 'lte': '2016-12-31' }, paginate=True ) 
#df.to_csv(path_or_buf="/Users/andrewplaate/mlp3/10Stocks.csv")

In [2]:
import gym
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import gym_trading

env = gym.make('trading-v0')
#env.time_cost_bps = 0 # 

env = env.unwrapped



import tensorflow as tf
import policy_gradient

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: Environment '<class 'gym_trading.envs.trading_env.TradingEnv'>' has deprecated methods. Compatibility code invoked.


/Users/colinsmith/miniconda3/envs/personal/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
INFO:policy_gradient:policy_gradient logger started.


In [5]:
# create the tf session
sess = tf.InteractiveSession()

# Input
num_actions=10  # same as # of stocks
Variables=3
obs_dim=num_actions*Variables
NumOfLayers=3
architecture = "LSTM" # right now, valid inputs are LSTM and FFNN
LR="Adam"
actFunc="relu"
regulizer="l2"
regulizerScale=0.01
avgfilename="randomfilename.p"


pg = policy_gradient.PolicyGradient(sess, obs_dim=obs_dim, 
                                    num_actions=num_actions,
                                    NumOfLayers=NumOfLayers, 
                                    LR=LR,
                                    architecture=architecture,
                                    actFunc=actFunc,
                                    learning_rate=1e-2,
                                    regulizer =regulizer,
                                    regulizerScale=regulizerScale,
                                    avgfilename=avgfilename
                                   )

# and now let's train it and evaluate its progress.  NB: this could take some time...
direc="/Users/colinsmith/mlp3"
load_model=False
#grads, grads_clipped = pg.get_grads_and_clipping()
'''with tf.Session() as sess:
    sess.run(init)  # actually running the initialization op
    _grads_clipped, _grads = sess.run(
                        [grads_clipped, grads],
                        feed_dict={pg.X: epX, pg._tf_epr: epr, pg._tf_y: epy, pg._tf_x: epx})'''
df,sf = pg.train_model(env, episodes=5, log_freq=1, load_model=False,model_dir = direc)#, load_model=True)


INFO:policy_gradient:year #     0, mean reward:  -1.3532, sim ret:   0.0000, mkt ret:   0.0000, net:   0.0000


[ 0.6144949]


INFO:policy_gradient:year #     1, mean reward:  -0.1522, sim ret:   0.0000, mkt ret:   0.0000, net:   0.0000


[ 0.6144949   0.01875039]


INFO:policy_gradient:year #     2, mean reward:   0.2612, sim ret:   0.0000, mkt ret:   0.0000, net:   0.0000


[ 0.6144949   0.01875039 -0.10553088]


INFO:policy_gradient:year #     3, mean reward:  -3.3050, sim ret:   0.0000, mkt ret:   0.0000, net:   0.0000


[ 0.6144949   0.01875039 -0.10553088  0.00828598]


INFO:policy_gradient:year #     4, mean reward:   0.0459, sim ret:   0.0000, mkt ret:   0.0000, net:   0.0000


[ 0.6144949   0.01875039 -0.10553088  0.00828598  0.08665886]


# Testing

In [ ]:
import gym
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import gym_trading

env1 = gym.make('testing-v0')
#env.time_cost_bps = 0 # 

env1 = env1.unwrapped



import tensorflow as tf
import policy_gradient

In [ ]:
# create the tf session
sess = tf.InteractiveSession()

# Input
num_actions=3  # same as # of stocks
Variables=3
obs_dim=num_actions*Variables
NumOfLayers=3
LR="Adam"
actFunc="relu"
regulizer="l2"
regulizerScale=0.01


pg = policy_gradient.PolicyGradient(sess, obs_dim=obs_dim, 
                                    num_actions=num_actions,
                                    NumOfLayers=NumOfLayers, 
                                    LR=LR,
                                    actFunc=actFunc,
                                    learning_rate=1e-2,
                                    regulizer =regulizer,
                                    regulizerScale=regulizerScale
                                   )

# and now let's train it and evaluate its progress.  NB: this could take some time...
direc="/Users/andrewplaate/mlp3/SavedModels/"
load_model=True
df,sf = pg.test_model(env1, episodes=1, log_freq=100, load_model=load_model,model_dir = direc)#, load_model=True)


### Results

Policy gradients beat the trading game!  That said, it doesn't work every time and it seems, looking at the charts below, as though it's a bit of a lucky thing.  But luck counts in the trading game as in life!


In [ ]:
sf['net'] = sf.simror - sf.mktror
#sf.net.plot()
sf.net.expanding().mean().plot()
sf.net.rolling(100).mean().plot()

In [ ]:
sf.net.rolling(100).mean().tail()